# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

# 1. Make Detections

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 3. Calculate Angles

In [4]:
shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
mouth_l = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].z]
mouth_r = [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].z]
nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,landmarks[mp_pose.PoseLandmark.NOSE.value].z]

In [25]:
def calculate_deviation(shoulder_l,shoulder_r,mouth_l,mouth_r,nose,base_shoulder_l,base_shoulder_r,base_mouth_l,base_mouth_r,base_nose):
    nose_f = abs(base_nose[0] - nose[0]) + abs(base_nose[1] - nose[1]) + abs(
            base_nose[2] - nose[2])
    mouth_l_f = abs(base_mouth_l[0]- mouth_l[0]) + abs(
        base_mouth_l[1]- mouth_l[1]) + abs(base_mouth_l[2]- mouth_l[2])
    mouth_r_f = abs(base_mouth_r[0]- mouth_r[0]) + abs(
        base_mouth_r[1]- mouth_r[1]) + abs(base_mouth_r[2]- mouth_r[2])
    shoulder_l_f = abs(base_shoulder_l[0]- base_shoulder_l[0]) + abs(
        base_shoulder_l[1]- shoulder_l[1]) + abs(base_shoulder_l[2]- shoulder_l[2])
    shoulder_r_f = abs(base_shoulder_r[0]- base_shoulder_r[0]) + abs(
        base_shoulder_r[1]- shoulder_r[1]) + abs(base_shoulder_r[2]- shoulder_r[2])
    deviation = int(
        ((nose_f + mouth_l_f + mouth_r_f + shoulder_l_f + shoulder_r_f) / (base_nose[0]+
         base_mouth_l[0] + base_mouth_r[0] + base_shoulder_l[0] +
         base_shoulder_r[0]) * 10))
    return deviation     

# 4. DEVIATION Counter

In [31]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            mouth_l = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].z]
            mouth_r = [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].z]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,landmarks[mp_pose.PoseLandmark.NOSE.value].z]
            if counter == 0:
                base_shoulder_r = shoulder_r
                base_shoulder_l = shoulder_l
                base_mouth_l = mouth_l
                base_mouth_r = mouth_r
                base_nose = nose
            # Calculate angle
            deviation = calculate_deviation(shoulder_l, shoulder_r,mouth_l,mouth_r,nose,base_shoulder_l,base_shoulder_r,base_mouth_l,base_mouth_r,base_nose)
            # Visualize angle
            cv2.putText(image, str(deviation),  
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
                                  
        except:
            pass
        stage = "great"
        if deviation > 10:
            stage = "stop slouching"
        if deviation > 20:
            stage="Stop sleeping"
        if deviation > 30:
            stage = "You are sleeping right?" 
        counter +=1
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (125,249,255), -1)
        
        # Rep data
        s4 = str(deviation) + str(stage)
        cv2.putText(image, s4, (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
       
        
        # Stage data
        print(deviation)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
2
2
2
2
2
1
1
2
2
2
1
1
1
1
4
3
4
5
4
4
2
2
2
3
3
5
4
4
4
4
3
2
2
2
3
5
5
4
5
4
4
3
3
4
4
4
5
5
5
5
5
5
4
4
4
5
6
6
6
6
5
3
2
3
3
3
3
3
3
2
2
2
2
3
3
3
3
3
3
3
3
3
5
4
4
4
5
4
3
3
3
4
5
5
5
5
5
4
4
3
3
4
6
6
5
5
5
5
5
5
5
5
5
5
5
5
6
5
4
4
4
4
4
4
4
4
4
5
4
3
3
2
3
2
2
2
2
2
1
1
1
1
1
1
2
2
3
3
3
3
3
3
3
3
3
5
5
6
5
5
5
6
6
6
5
4
5
5
6
6
7
6
6
6
6
7
7
7
7
8
7
7
8
9
6
7
8
8
9
10
12
13
13
14
15
15
14
14
14
13
15
14
15
15
12
11
10
5
6
8
16
16
15
19
21
24
24
23
27
25
25
24
23
18
18
17
18
19
15
15
16
21
15
16
19
18
18
16
14
14
15
15
15
16
16
16
15
14
14
13
13
12
12
11
10
11
9
10
11
15
23
25
29
39
37
42
41
37
34
31
19
5
5
4
6
8
4
15
18
6
7
8
17
25
17
26
30
28
33
40
36
31
35
34
30
27
16
8
10
5
9
6
5
9
9
10
11
11
12
12
12
12
13
12
12
14
15
18
21
22
28
31
28
33
34
36
33
34
37
37
39
40
38
44
38
38
38
43
42
41
37
35
35
36
36
35
34
32
33
33
32
36
34
35
35
35
34
34
31
30
32
31
32
28
27
36
30
28
34
33
29
24
20
25
15
15
6
6
4
5
8
6
5
4

KeyboardInterrupt: 